In [ ]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
import datetime
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

import re

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import StaleElementReferenceException

# Dentro do while, após abrir o site:
wait = WebDriverWait(driver, 10)

data_de_hoje = datetime.date.today()


base_url = "https://mercado.carrefour.com.br/s?q={produto}&sort=score_desc&page=0"

produtos = ["frango", "picanha", "tilapia", "salmao", "bacalhau", "ovos", "leite", "azeite"]

for produto in produtos:
    carrefour  = base_url.format(produto=produto)
    service = ChromeService()
    # Iniciar o driver
    driver = webdriver.Chrome(service=service)
    #driver  = webdriver.Chrome(service=service)
    driver.get(carrefour)

    time.sleep(4)
    driver.find_element(By.CSS_SELECTOR, r"#__next > div:nth-child(5) > div.bg-white.min-h-\[39px\].border-b.border-gray-light > div > div > div").click()
    time.sleep(2)
    driver.find_element(By.XPATH, "/html/body/div[4]/div[2]/div[2]/div/div/div[2]/div[2]/div/section/div/div[1]/button[2]").click()
    time.sleep(4)
    select = Select(driver.find_element(By.CSS_SELECTOR, "#selectCity"))
    select.select_by_index(64)

    time.sleep(4)
    botao_cidade = driver.find_elements(By.CSS_SELECTOR, "div[class='grid grid-flow-col']")
    loja, _, cidade, _ = botao_cidade[0].text.split("\n")
    botao_cidade[0].click()

    time.sleep(5)
    for i in range(10):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    divs_carrefour = driver.find_elements(By.CSS_SELECTOR, "section[data-product-card-content='true']")

    for div in divs_carrefour:
        dtxt = div.text
        if len(dtxt) > 0:
            precos_carrefour.append(dtxt + " | " + loja + " | " + cidade)

    driver.quit()



##### ORGANIZACAO DOS DADOS #####
dados_brutos = precos_carrefour

dados_estruturados = []

for linha in dados_brutos:
    try:
        produto_bloco, loja, endereco = linha.split(" | ")
        linhas = produto_bloco.strip().split("\n")

        # Heurística com base no número de linhas
        if len(linhas) >= 4:
            produto_nome = next((x for x in linhas if "R$" not in x and "%" not in x and "promoção" not in x), "").strip()
            precos_encontrados = [x for x in linhas if re.match(r"R\$ ?\d+,\d{2}", x.strip())]
            preco_final = precos_encontrados[-1].strip() if precos_encontrados else None
            desconto = next((x for x in linhas if "%" in x and "-" in x), None)
        else:
            produto_nome = next((x for x in linhas if "R$" not in x), "").strip()
            preco_final = next((x for x in linhas if "R$" in x), None)
            desconto = None

        cidade_estado = endereco.split(",")[-1].strip()
        cidade, estado = cidade_estado.split(" - ")

        preco_final = float(preco_final.replace("R$", "").replace(",", ".").strip()) if preco_final else None
        desconto = float(desc.replace("%", "").replace(",", ".").strip()) if (desc := desconto) else None

        dados_estruturados.append({
            "produto": produto_nome,
            "preco": preco_final,
            "desconto": desconto,
            "loja": loja.strip(),
            "endereco": endereco.strip(),
            "cidade": cidade.strip(),
            "estado": estado.strip()
        })

    except Exception as e:
        print(f"Erro ao processar linha: {linha}\n-> {e}")

df_produtos = pd.DataFrame(dados_estruturados)

# import ace_tools as tools; tools.display_dataframe_to_user(name="Produtos Carrefour Estruturados", dataframe=df_produtos)
df_produtos 


##### INCLUIR A COLUNA DATA ######
df_produtos['data'] = data_de_hoje
# Reorganizar explicitamente as colunas com 'data' na frente
colunas = ['data'] + [col for col in df_produtos.columns if col != 'data']
df_produtos = df_produtos[colunas]


# Ler o arquivo CSV salvo anteriormente
df_produtos_anterior = pd.read_csv("precos_carrefour_morumbi.csv")

df_produtos = pd.concat([df_produtos_anterior, df_produtos], ignore_index=True)
df_produtos = df_produtos.drop_duplicates()

df_produtos.to_csv('precos_carrefour_morumbi.csv', index=False)

